In [1]:
!pip install -r https://raw.githubusercontent.com/datamllab/automl-in-action-notebooks/master/requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.8 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11
ERROR: Could not find a version that satisfies the requirement tensorflow==2.10.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0r

# RNN
In this section, we will introduce how to use recurrent neural networks for text
classification.
The dataset we use is the IMDB Movie Reviews.
We use the reviews written by the users as the input and try to predict whether the they
are positive or negative.

## Preparing the data

You can use the following code to load the IMDB dataset.


In [2]:
import tensorflow as tf

tf.random.set_seed(42)

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

max_words = 10000
embedding_dim = 32

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=max_words
)
print(train_data.shape)
print(train_labels.shape)
print(train_data[:2])
print(train_labels[:2])


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(25000,)
(25000,)
[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345

The code above would load the reviews into train_data and test_data, load the labels
(positive or negative) into train_labels and test_labels. As you can see the reviews in
train_data are lists of integers instead of texts. It is because the raw texts cannot be
used as an input to a neural network. Neural networks only accepts numerical data as
inputs.

The integers we see above is the raw text data after a preprocessing step named
tokenization. It first split each review into a list of words and assign an integer to
each of the words. For example, a scentence "How are you? How are you doing?" will be
transformed into a list of words as ["how", "are", "you", "how", "are", "you", "doing"].
Then transformed to [5, 8, 9, 5, 8, 9, 7]. The integers doesn't have special meanings but
a representation of the words. Same integers represents the same words, different
integers represents different words.

The labels are also integers, where 1 represents positive, 0 represents negative.

Then, we pad the data to the same length.


In [3]:
# Pad the sequence to length max_len.
maxlen = 100
print(len(train_data[0]))
print(len(train_data[1]))
train_data = sequence.pad_sequences(train_data, maxlen=maxlen)
test_data = sequence.pad_sequences(test_data, maxlen=maxlen)
print(train_data.shape)
print(train_labels.shape)


218
189
(25000, 100)
(25000,)


## Building your network
The next step is to build your neural network model and train it.
We will introduce the neural network in three steps.
The first step is the embedding, which transform each integer list into a list of vectors.
The second step is to feed the vectors to the recurrent neural network.
The third step is to use the output of the recurrent neural network for classification.
### Embedding
Embedding means find a corresponding numerical vector for each word, which is now an
integer in the list.
The numerical vector can be seen as the coordinate of a point in the space.
We embed the words into specific points in the space.
That is why we call the process embedding.

To implement it, we use a Keras Embedding layer.
First, we need to create a Keras Sequential model.
Then, we add the layers one by one to the model.
The order of the layers is from the input to the output.


In [4]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras import Sequential

max_words = 10000
embedding_dim = 32

model = Sequential()
model.add(Embedding(max_words, embedding_dim))
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In the code above, we initialized an Embedding layer.
The max_words is the vocabulary size, which is an integer meaning how many different
words are there in the input data.
The integer 16 means the length of the vector representation fro each word is 32.
The output tensor of the Embedding layer is (batch_size, max_len, embedding_dim).
### Recurrent Neural Networks
After the embedding layer, we need to use a recurrent neural network for the
classification.
Recurrent neural networks can handle sequential inputs.
For example, we input a movie review as a sequence of word embedding vectors to it, which
are the output of the embedding layer.
Each vector has a length of 32.
Each review contains 100 vectors.
If we see the RNN as a whole, it takes 100 vectors of length 16 altogether.
However, in the real case, it takes one vector at a time.

Each time the RNN takes in a word embedding vector,
it not only takes the word embedding vector, but another state vector as well.
You can think the state vector as the memory of the RNN.
It memorizes the previous words it taken as input.
In the first step, the RNN has no previous words to remember.
It takes an initial state, which is usually empty,
and the first word embedding vector as input.
The output of the first step is actually the state to be input to the second step.
For the rest of the steps, the RNN will just take the previous output and the current
input as input,
and output the state for the next step.
For the last step, the output state is the final output we will use for the
classification.

We can use the following python code to illustrate the process.
```python
state = [0] * 32
for i in range(100):
    state = rnn(embedding[i], state)
return state
```
The returned state is the final output of the RNN.

Sometimes, we may also need to collect the output of each step as shown in the following
code.
```python
state = [0] * 32
output = []
for i in range(100):
    state = rnn(embedding[i], state)
    output.append(state)
return output
```
In the code above, the output of an RNN can also be a sequence of vectors, which is the
same format as the input to the RNN.
Therefore, we can make the RNN deeper by stacking multiple RNN layers together.


To implement the RNN described, we need the SimpleRNN layer in Keras.


In [5]:
from tensorflow.keras.layers import SimpleRNN

model.add(SimpleRNN(embedding_dim, return_sequences=True))
model.add(SimpleRNN(embedding_dim, return_sequences=True))
model.add(SimpleRNN(embedding_dim, return_sequences=True))
model.add(SimpleRNN(embedding_dim))
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_3 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

The return_sequences parameter controlls whether to collect all the output vectors of an
RNN or only collect the last output. It is set to False by default.
### Classification Head
Then we will use the output of the last SimpleRNN layer, which is a vector of length 32,
as the input to the classification head.
In the classification head, we use a fully-connected layer for the classification.
Then we compile and train the model.


In [6]:
from tensorflow.keras.layers import Dense

model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", metrics=["acc"], loss="binary_crossentropy")
model.fit(train_data, train_labels, epochs=2, batch_size=128)


Epoch 1/2
196/196 ━━━━━━━━━━━━━━━━━━━━ 32s 124ms/step - acc: 0.5115 - loss: 0.7025
Epoch 2/2
196/196 ━━━━━━━━━━━━━━━━━━━━ 24s 124ms/step - acc: 0.6408 - loss: 0.6317


Then we can validate our model on the testing data.


In [7]:
model.evaluate(test_data, test_labels)


782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - acc: 0.8106 - loss: 0.4227


[0.41919928789138794, 0.8136399984359741]

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 32)             │         320,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 100, 32)             │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 100, 32)             │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (None, 100, 32)             │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_3 (SimpleRNN)             │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 985,061 (3.76 MB)

 Trainable params: 328,353 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 656,708 (2.51 MB)

In [29]:
!pip install --upgrade --quiet keras-hub-nightly keras-nightly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.1/699.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00


In [37]:
# https://colab.research.google.com/github/keras-team/keras-io/blob/master/guides/ipynb/keras_hub/getting_started.ipynb#scrollTo=GU2ttYiYw9Hz
import keras_hub
causal_lm = keras_hub.models.CausalLM.from_preset("gpt2_base_en")
tokenizer = causal_lm.preprocessor.tokenizer
tokens_ids = tokenizer.tokenize("The quick brown fox jumps over the lazy dog.")
data=list(tokens_ids.numpy())



[464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]
The quick brown fox jumps over the lazy dog.


Let's try to use this model.

In [38]:
sentence = "I hate when people tell what to do!"
import keras_hub
causal_lm = keras_hub.models.CausalLM.from_preset("gpt2_base_en")
tokenizer = causal_lm.preprocessor.tokenizer
data = tokenizer.tokenize(sentence)
print(list(tokens_ids.numpy()))


def pad_sequences(self, data, maxlen=5000):
    result = keras.preprocessing.sequence.pad_sequences(data,
                                                    value=0,
                                                    padding='post',
                                                    maxlen=maxlen)

#x = np.array([1, 2, 3])
# Reshape the input to the required sample_size x time_steps x features
#x_input = np.reshape(x,(1, 3, 1))
#y_pred_model = model.predict(x_input)
#print(y_pred_model)

[464, 2068, 7586, 21831, 18045, 625, 262, 16931, 3290, 13]
